In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, precision_score, recall_score

In [2]:
data = pd.read_csv("covidpred-preprocessed.csv")

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136294 entries, 0 to 136293
Data columns (total 10 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Unnamed: 0           136294 non-null  int64  
 1   cough                136294 non-null  float64
 2   fever                136294 non-null  float64
 3   sore_throat          136294 non-null  float64
 4   shortness_of_breath  136294 non-null  float64
 5   head_ache            136294 non-null  float64
 6   corona_result        136294 non-null  int64  
 7   age_60_and_above     136294 non-null  int64  
 8   gender               136294 non-null  int64  
 9   test_indication      136294 non-null  int64  
dtypes: float64(5), int64(5)
memory usage: 10.4 MB


In [4]:
data.head()

,Unnamed: 0,cough,fever,sore_throat,shortness_of_breath,head_ache,corona_result,age_60_and_above,gender,test_indication
0,122808,1.0,0.0,0.0,0.0,0.0,0,1,1,2
1,122809,1.0,0.0,0.0,0.0,0.0,1,0,0,2
2,122810,0.0,0.0,0.0,0.0,0.0,0,0,0,2
3,122811,0.0,1.0,0.0,0.0,0.0,0,0,0,0
4,122812,1.0,0.0,0.0,0.0,0.0,0,1,0,2


In [5]:
data.drop(["Unnamed: 0"], axis=1, inplace=True)

In [6]:
data.head()

,cough,fever,sore_throat,shortness_of_breath,head_ache,corona_result,age_60_and_above,gender,test_indication
0,1.0,0.0,0.0,0.0,0.0,0,1,1,2
1,1.0,0.0,0.0,0.0,0.0,1,0,0,2
2,0.0,0.0,0.0,0.0,0.0,0,0,0,2
3,0.0,1.0,0.0,0.0,0.0,0,0,0,0
4,1.0,0.0,0.0,0.0,0.0,0,1,0,2


In [7]:
data[data.corona_result == 0].shape

(125668, 9)

In [8]:
data[data.corona_result == 1].shape

(10626, 9)

## Dataset is highly unbalanced

##  Resampling the class with corona_result = 1 into 8 different datasets and training them on 8 models and then using ensemble method

In [9]:
data0 = data[data.corona_result == 0]
data1 = data[data.corona_result == 1]

In [10]:
data01 = data0[:15000]
data02 = data0[15000:30000]
data03 = data0[30000:45000]
data04 = data0[45000:60000]
data05 = data0[60000:75000]
data06 = data0[75000:90000]
data07 = data0[90000:105000]
data08 = data0[105000:]

In [11]:
data0 = [data01, data02, data03, data04, data05, data06, data07, data08]
dataCombined = []
for data in data0:
    tempFrames = [data, data1]
    dataCombined.append(pd.concat(tempFrames))

In [12]:
dataCombined[0][dataCombined[0].corona_result == 0].shape

(15000, 9)

In [13]:
dataCombined[0][dataCombined[0].corona_result == 1].shape

(10626, 9)

In [20]:
y = []
for data in dataCombined:
    y.append(data["corona_result"].to_numpy())

In [22]:
x = []
for data in dataCombined:
    temp = data.drop(["corona_result"], axis=1)
    x.append(temp.to_numpy())

## Gradient Boosting

In [27]:
models = []
total_accuracy = []
total_f1 = []
total_precision = []
total_recall = []
for i in range(8):
    x_train, x_test, y_train, y_test = train_test_split(x[i], y[i], test_size=0.10)
    xgb_classifier = xgb.XGBClassifier(max_depth=7, learning_rate=0.008, objective='multi:softprob', 
                                   n_estimators=600, sub_sample=0.8, num_class=2,
                                   booster='gbtree', n_jobs=4)
    xgb_classifier.fit(x_train, y_train)
    pred = xgb_classifier.predict(x_test)
    models.append(xgb_classifier)
    accuracy = accuracy_score(y_test, pred)
    total_accuracy.append(accuracy)
    f1 = f1_score(y_test, pred, average='macro') 
    prec = precision_score(y_test, pred, average='macro') 
    rec = recall_score(y_test, pred, average='macro')
    total_f1.append(f1)
    total_precision.append(prec)
    total_recall.append(rec)

C:\Users\ritvi\AppData\Roaming\Python\Python39\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[23:07:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "sub_sample" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[23:07:59] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\ritvi\AppData\Roaming\Python\Python39\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[23:08:13] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "sub_sample" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[23:08:13] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\ritvi\AppData\Roaming\Python\Python39\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

[23:09:11] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "sub_sample" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[23:09:11] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\ritvi\AppData\Roaming\Python\Python39\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[23:09:26] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "sub_sample" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[23:09:26] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\ritvi\AppData\Roaming\Python\Python39\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[23:09:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "sub_sample" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[23:09:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [28]:
from statistics import mean
print("Total accuracy: ", mean(total_accuracy))
print("Total f1 score: ", mean(total_f1))
print("Total precision: ", mean(total_precision))
print("Total recall: ", mean(total_recall))

Total accuracy:  0.8823798114978578
Total f1 score:  0.874545049017958
Total precision:  0.8876164440126997
Total recall:  0.8676595472336271


## DNN

In [29]:
from tensorflow import keras

In [30]:
models_dnn = []
total_accuracy = []
total_f1 = []
total_precision = []
total_recall = []
for i in range(8):
    x_train, x_test, y_train, y_test = train_test_split(x[i], y[i], test_size=0.10)
    y_train_dnn = np.zeros((y_train.shape[0], 2))
    y_test_dnn = np.zeros((y_test.shape[0], 2))

    for i in range(y_train.shape[0]):
        if y_train[i] == 0:
            y_train_dnn[i][0] = 1
        else:
            y_train_dnn[i][1] = 1
    for i in range(y_test.shape[0]):
        if y_test[i] == 0:
            y_test_dnn[i][0] = 1
        else:
            y_test_dnn[i][1] = 1
    model = keras.models.Sequential()
    model.add(keras.layers.Dense(256))
    model.add(keras.layers.Activation('relu'))
    model.add(keras.layers.Dense(64))
    model.add(keras.layers.Activation('relu'))
    model.add(keras.layers.Dense(2))
    model.add(keras.layers.Activation('softmax'))
    opt = keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(loss='binary_crossentropy', optimizer=opt,metrics=['accuracy'])
    model.fit(x_train, y_train_dnn, batch_size=64, epochs=10)
    original_pred = model.predict(x_test)
    pred = np.zeros((original_pred.shape[0]))
    for i in range(original_pred.shape[0]):
        if original_pred[i][0] > original_pred[i][1]:
            pred[i] = 0
        else:
            pred[i] = 1
    models_dnn.append(model)
    accuracy = accuracy_score(y_test, pred)
    total_accuracy.append(accuracy)
    f1 = f1_score(y_test, pred, average='macro') 
    prec = precision_score(y_test, pred, average='macro') 
    rec = recall_score(y_test, pred, average='macro')
    total_f1.append(f1)
    total_precision.append(prec)
    total_recall.append(rec)

Epoch 1/10
361/361 [==============================] - 2s 2ms/step - loss: 0.5631 - accuracy: 0.7081
Epoch 2/10
361/361 [==============================] - 1s 2ms/step - loss: 0.4353 - accuracy: 0.7963
Epoch 3/10
361/361 [==============================] - 1s 2ms/step - loss: 0.3867 - accuracy: 0.8335
Epoch 4/10
361/361 [==============================] - 1s 2ms/step - loss: 0.3634 - accuracy: 0.8481
Epoch 5/10
361/361 [==============================] - 1s 2ms/step - loss: 0.3509 - accuracy: 0.8630
Epoch 6/10
361/361 [==============================] - 1s 2ms/step - loss: 0.3435 - accuracy: 0.8648
Epoch 7/10
361/361 [==============================] - 1s 2ms/step - loss: 0.3395 - accuracy: 0.8670
Epoch 8/10
361/361 [==============================] - 1s 2ms/step - loss: 0.3367 - accuracy: 0.8685
Epoch 9/10
361/361 [==============================] - 1s 2ms/step - loss: 0.3354 - accuracy: 0.8684
Epoch 10/10
361/361 [==============================] - 1s 2ms/step - loss: 0.3339 - accuracy: 0.8690

In [32]:
from statistics import mean
print("Total accuracy: ", mean(total_accuracy))
print("Total f1 score: ", mean(total_f1))
print("Total precision: ", mean(total_precision))
print("Total recall: ", mean(total_recall))

Total accuracy:  0.8820638441622549
Total f1 score:  0.8739643010034325
Total precision:  0.8871736895641218
Total recall:  0.8669069904393463
